# Extract MODIS Site Data and Generate Samples
## MODIS Site Data
For each site in the LFMC sample data, extract the full time series of MODIS reflectance and snow-cover data, and save to CSV files. Note: if the output csv files already exist they are assumed to be correct and are not over-written. The data is gap-filled after being saved but before being used to create the MODIS sample data.

## MODIS Sample Data
For each sample, extract the timeseries MODIS reflectance data. The timeseries length is determined by the MODIS_TS_LENGTH value. The sample is rejected if the full timeseries cannot be extracted (start/end outside the full site time series). The extracted MODIS data is combined to a single dataframe and saved. Another LFMC sample dataset containing only the valid samples is also created.

## Input Files
- `LFMC_sites.csv` and `samples_365days.csv` created by the `Extract Auxiliary Data.ipynb` and `Extract ERA5 DATA.ipynb` notebooks.

## Output Files
- The extracted sites data are created in `MODIS_DIR` (by default `MCD43A4` located in `LFMC_DATA_DIR/GEE_DIR`). The directory will contain a CSV file for each site.
- The extracted MODIS data for each samples and updated samples data are created in `DATASETS_DIR`. File names include the time series length (i.e. 730days) of the extracted MODIS data. So, with the default settings they are `modis_730days.csv` and `samples_730days.csv`.

## Notes
1. This notebook should be run after running the `Extract Auxiliary Data.ipynb` and `Extract ERA5 DATA.ipynb` notebooks.
2. The generated name for `GEE_DIR` includes the projection and scale of the extracted GEE data. 
3. It will take about 8.5 hours to run if there are no existing site extracts.
4. Data for 67724 samples will be extracted. If the samples input is the ERA5 extract sample output, there should be no invalid or discarded samples.
5. There should be no invalid sites, but occasionally extraction from GEE will fail for a site. If this happens re-run the notebook (keep the existing site CSV files so they are not re-extracted).

In [1]:
import os
os.environ["HDF5_DISABLE_VERSION_CHECK"] = "1"
import numpy as np
import pandas as pd
import re
import time
from datetime import datetime
from datetime import timedelta

import initialise
import common
from data_extract_utils import extract_timeseries_data, sort_key
from timeseries_extractor import GeeTimeseriesExtractor

### Program parameters and constants

GEE Parameters
- Reflectance product is MCD43A4 - daily reflectance using 8-day composites
- Scale set to use native MODIS resolution
- Extracts two years' of data for each sample to allow for a 1-year time series with a 1-year lead time

In [2]:
START_DATE = "2000-03-01"
END_DATE = "2019-01-01"  # Final day retrieved will be 2018-12-31

# MODIS time series constants
MODIS_TS_LENGTH = 365
MODIS_TS_OFFSET = 1
MODIS_TS_FREQ = 1

# MODIS data details
PRODUCT = "MODIS/006/MCD43A4"
BANDS = ["Nadir_Reflectance_Band1",
         "Nadir_Reflectance_Band2",
         "Nadir_Reflectance_Band3",
         "Nadir_Reflectance_Band4",
         "Nadir_Reflectance_Band5",
         "Nadir_Reflectance_Band6",
         "Nadir_Reflectance_Band7"]
# Extract data using default MODIS scale and projection
SCALE = 463.3127
PROJ = "SR-ORG:6974"

EARLIEST_SAMPLE = datetime.strptime(START_DATE, '%Y-%m-%d') + timedelta(
    days=MODIS_TS_LENGTH * MODIS_TS_FREQ + MODIS_TS_OFFSET  - 1)

### Directories and Files

In [3]:
# Sub-directories for GEE extracts
GEE_DIR = f"GEE_{common.PROJ.replace(':', '-')}_{int(common.SCALE)}"
MODIS_DIR = os.path.join(common.LFMC_DATA_DIR, GEE_DIR, "MCD43A4")

# File Names
SAMPLES_INPUT = os.path.join(common.DATASETS_DIR, f"samples_{MODIS_TS_LENGTH * MODIS_TS_FREQ}days.csv")
SAMPLES_OUTPUT = os.path.join(common.DATASETS_DIR, f"samples_{MODIS_TS_LENGTH * MODIS_TS_FREQ}days_new.csv")
MODIS_OUTPUT = os.path.join(common.DATASETS_DIR, f"modis_{MODIS_TS_LENGTH * MODIS_TS_FREQ}days.csv")

# Create output directories if necessary
if not os.path.exists(MODIS_DIR):
    os.makedirs(MODIS_DIR)

## Main Processing
Connect to GEE

In [4]:
import ee
ee.Initialize()

### Generate MODIS sample data

For each site, get the sample data for each sample at the site

Note: This gets the data using the default MODIS scale and projection. To change the scale and/or projection, add calls to the `GeeTimeseriesExtractor.setProjScale` method

In [5]:
sites = pd.read_csv(common.LFMC_SITES, float_precision="high")
samples = pd.read_csv(SAMPLES_INPUT, float_precision="high")
modis_extractor = GeeTimeseriesExtractor(PRODUCT, BANDS, START_DATE, END_DATE,
                                         dir_name=MODIS_DIR)
modis_data, valid_data, invalid_pixels, invalid_sites = extract_timeseries_data(
    modis_extractor, sites, samples, EARLIEST_SAMPLE,
    MODIS_TS_OFFSET, MODIS_TS_LENGTH, MODIS_TS_FREQ)

# modis_data = []
# valid_data = [False] * samples.shape[0]
# invalid_pixels = []
# invalid_sites = []
# for site_idx, site in sites.iterrows():
#     print(f'Processing site {site.Site}')
#     site_samples = samples[samples.Site == site.Site]
#     try:
#         modis_df = modis_extractor.get_and_save_data(site)
#     except:
#         print(f'Failed to extract data for {site.Site}')
#         invalid_sites.append(site.Site)
#         continue
#     for index, sample in site_samples.iterrows():
#         sample_data = get_sample_data(sample["Sampling date"], modis_df,
#                                       MODIS_TS_OFFSET, MODIS_TS_LENGTH, MODIS_TS_FREQ)
#         if sample_data is None or np.isnan(sample_data.sum()):
#             invalid_pixels.append(index)
#         else:
#             modis_data.append([sample.ID] + list(sample_data))
#             valid_data[index] = True

Processing site C4_1
Processing site C4_2
Processing site C4_3
Processing site C4_4
Processing site C4_5
Processing site C6_1
Processing site C6_2
Processing site C6_3
Processing site C6_4
Processing site C6_5
Processing site C6_6
Processing site C6_7
Processing site C6_8
Processing site C6_9
Processing site C6_10
Processing site C6_11
Processing site C6_12
Processing site C6_13
Processing site C6_14
Processing site C6_16
Processing site C6_17
Processing site C6_18
Processing site C6_19
Processing site C6_20
Processing site C6_21
Processing site C6_22
Processing site C6_23
Processing site C6_24
Processing site C6_25
Processing site C6_26
Processing site C6_27
Processing site C6_28
Processing site C6_29
Processing site C6_30
Processing site C6_31
Processing site C6_32
Processing site C6_33
Processing site C6_34
Processing site C6_35
Processing site C6_36
Processing site C6_37
Processing site C6_38
Processing site C6_39
Processing site C6_40
Processing site C6_41
Processing site C6_42
Pr

Summary of sites/samples not extracted

In [6]:
print(f'Invalid sites: {len(invalid_sites)}; Invalid pixels: {len(invalid_pixels)}; ')
print(invalid_sites)
print(invalid_pixels)

Invalid sites: 0; Invalid pixels: 0; 
[]
[]


### Save Results
Save and display sample reflectance data

In [7]:
modis_data = pd.DataFrame(modis_data)
ts_days = (MODIS_TS_LENGTH - 1) * MODIS_TS_FREQ
modis_data.columns = ["ID"] + [f'{day-MODIS_TS_OFFSET:04}_{band+1}'
                               for day in range(-ts_days, 1, MODIS_TS_FREQ)
                               for band in range(len(BANDS))]
modis_data.sort_values('ID', inplace=True, key=lambda x: x.apply(sort_key))
modis_data.to_csv(MODIS_OUTPUT, index=False)
modis_data

,ID,-365_1,-365_2,-365_3,-365_4,-365_5,-365_6,-365_7,-364_1,-364_2,...,-002_5,-002_6,-002_7,-001_1,-001_2,-001_3,-001_4,-001_5,-001_6,-001_7
0,C4_1_1,1760,2497,1060,1461,3243,3128,2848,1752,2485,...,3285,3182,2553,1701,2738,900,1408,3285,3203,2568
1,C4_1_2,1757,2460,1060,1457,3251,3469,2965,1883,2640,...,3517,3526,2816,1820,2847,1015,1488,3506,3523,2816
2,C4_1_3,1966,2809,1124,1599,3497,3705,3108,1983,2826,...,3439,3521,2841,1896,2833,1036,1504,3452,3528,2847
3,C4_1_4,2003,2837,1147,1617,3407,3549,3155,2038,2891,...,3436,3480,2765,1849,2728,1039,1472,3404,3462,2754
4,C4_1_5,1929,2710,1135,1580,3368,3381,3080,1922,2710,...,3297,3368,2683,1815,2702,976,1420,3287,3326,2642
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67719,C13_4_14,490,1881,267,477,2053,1623,788,493,1888,...,2358,1922,981,652,2062,351,589,2421,1931,1005
67720,C13_4_15,538,1912,298,515,2163,1690,839,536,1907,...,2383,1840,948,625,2039,342,574,2369,1811,925
67721,C13_4_16,522,1846,287,491,2060,1665,801,524,1839,...,2294,1798,960,640,1977,345,562,2294,1805,961
67722,C13_4_17,500,1796,267,451,1953,1610,741,495,1785,...,2208,1492,830,601,1924,320,524,2197,1489,824


Save and display the valid samples

In [8]:
# valid_samples = samples[valid_data].sort_values('ID', key=lambda x: x.apply(sort_key))
# valid_samples.to_csv(SAMPLES_OUTPUT, index=False)
# valid_samples